In [4]:
import tensorflow as tf
import numpy as np

In [100]:
impact9x9=[
    [ 
        0x1 << c if r == 4 and c<4 
        else 0x1 << (c-1) if c>4 and r==4

        else 0x100 << c if c == 8-r and c<4
        else 0x100 << (c-1) if c == 8-r and c>4

        else 0x10000 << 8-r-1 if r<4 and c == 4  
        else 0x10000 << 8-r if r>4 and c==4

        else 0x1000000 << 8-c-1 if c == r and c<4
        else 0x1000000 << 8-c if c == r and c>4

        else 0
         for c in range(9) 
    ] for r in range(9) 
]
impact9x9

[[2147483648, 0, 0, 0, 8388608, 0, 0, 0, 32768],
 [0, 1073741824, 0, 0, 4194304, 0, 0, 16384, 0],
 [0, 0, 536870912, 0, 2097152, 0, 8192, 0, 0],
 [0, 0, 0, 268435456, 1048576, 4096, 0, 0, 0],
 [1, 2, 4, 8, 0, 16, 32, 64, 128],
 [0, 0, 0, 2048, 524288, 134217728, 0, 0, 0],
 [0, 0, 1024, 0, 262144, 0, 67108864, 0, 0],
 [0, 512, 0, 0, 131072, 0, 0, 33554432, 0],
 [256, 0, 0, 0, 65536, 0, 0, 0, 16777216]]

In [120]:
# l3 is the nw-line. point 2,2 on that line sees the stone at a distance of 2
x=impact9x9[2][2]
l0=(x & 0xFF)
l1=(x & 0xFF00) >> 8
l2=(x & 0xFF0000) >> 16
l3=(x & 0xFF000000) >> 24
[np.sign(l3 & (1<<i)) for i in range(8)]

[0, 0, 0, 0, 0, 1, 0, 0]

In [112]:
def stone_at(r,c, n=15):
    """
    Construct a complete nxn impact representation of a stone at row=r, col=c
    """
    src=np.hstack([
        np.zeros((n+8,c),dtype=np.int),
        np.vstack([
            np.zeros((r,9), dtype=np.int32), 
            impact9x9, 
            np.zeros((n-1-r,9), dtype=np.int32)
        ]),
        np.zeros((n+8,n-1-c),dtype=np.int)
    ])
    return (src[4:-4].T[4:-4].T).copy()

In [116]:
s77=stone_at(7,7, n=15)

In [117]:
np.shape(s77)

(15, 15)

In [118]:
np.log2(s77[3:-3].T[3:-3].T)

/Users/wgiersche/workspace/tutorials/other_stuff/venv3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log2
  """Entry point for launching an IPython kernel.


array([[ 31., -inf, -inf, -inf,  23., -inf, -inf, -inf,  15.],
       [-inf,  30., -inf, -inf,  22., -inf, -inf,  14., -inf],
       [-inf, -inf,  29., -inf,  21., -inf,  13., -inf, -inf],
       [-inf, -inf, -inf,  28.,  20.,  12., -inf, -inf, -inf],
       [  0.,   1.,   2.,   3., -inf,   4.,   5.,   6.,   7.],
       [-inf, -inf, -inf,  11.,  19.,  27., -inf, -inf, -inf],
       [-inf, -inf,  10., -inf,  18., -inf,  26., -inf, -inf],
       [-inf,   9., -inf, -inf,  17., -inf, -inf,  25., -inf],
       [  8., -inf, -inf, -inf,  16., -inf, -inf, -inf,  24.]])

In [111]:
import sys
sys.getsizeof(s77)

1912

In [136]:
n=15
impacts=[[stone_at(r,c,n) for c in range(n)] for r in range(n)]

In [137]:
(impacts[0][14])

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      1,      2,      4,      8,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,   2048, 524288],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,   1024,      0, 262144],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,    512,      0,      0, 131072],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,    256,      0,      0,      0,  65536],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0],
       [     0,      0,      0,      0,  

In [139]:
(impacts[0][14] | impacts[1][14])[1][13]

2056

That's 2048 (bit 3 on north east line) and 8 (bit 3 on east line)

The above expression answers the question what an observer on (1,13) sees when (0,14) and (1,14) are occupied:
It sees the two stones in one step each to north east and east respectively. That's what we expect.
```
      (0,14)
(1,13)(1,14)
```

So, the full impact of an ensemble of stones on the entire field is simply computed by or-ing the pre-computed impacts of each stone within that ensemble. That's orders of magnitude faster than the for-comprehension we used in the naive approach.

In [140]:
import tensorflow as tf

In [172]:
black=0x8A458133
white=0x41824284

In [173]:
# neighbourhoods black and white
nb = tf.constant(black, dtype=tf.int32)
nw = tf.constant(white, dtype=tf.int32)

In [174]:
nw_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 0))
n_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 8))
ne_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 16))
e_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 24))

nw_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 0))
n_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 8))
ne_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 16))
e_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 24))

In [175]:
with tf.Session() as s:
    print(n_w.eval())

66


In [181]:
keys=[256*b+w for b in range(256) for w in range(256)]
values=keys
table = tf.contrib.lookup.HashTable(
        tf.contrib.lookup.KeyValueTensorInitializer(keys, values),
        0)

In [188]:
with tf.Session():
    table.init.run()
    out=table.lookup(tf.constant(10))
    print(out.eval())

10


In [193]:
class TotalScore:

    def __init__(self, kappa1):

        keys, values = ...TODO!
        
        self.kappa1 = kappa1
        self.table = tf.contrib.lookup.HashTable(
            tf.contrib.lookup.KeyValueTensorInitializer(keys, values),0)
    
    def as_tensor(self, nb, nw):    
        nw_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 0))
        n_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 8))
        ne_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 16))
        e_b = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nb, 24))

        nw_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 0))
        n_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 8))
        ne_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 16))
        e_w = tf.bitwise.bitwise_and(0xFF, tf.bitwise.right_shift(nw, 24)) 

        score_nw_w = self.table.lookup(256*nw_w + nw_b)
        score_n_w = self.table.lookup(256*n_w + n_b)
        score_ne_w = self.table.lookup(256*ne_w + ne_b)
        score_e_w = self.table.lookup(256*e_w + e_b)

        score_nw_b = self.table.lookup(256*nw_b + nw_w)
        score_n_b = self.table.lookup(256*n_b + n_w)
        score_ne_b = self.table.lookup(256*ne_b + ne_w)
        score_e_b = self.table.lookup(256*e_b + e_w)

        ts_w = tf.add( 
            tf.pow(score_nw_w, tf.kappa1),
            tf.add (
                tf.pow(score_n_w, tf.kappa1),
                tf.add (
                    tf.pow(score_ne_w, tf.kappa1), 
                    score_e_w)))

        ts_b = tf.add( 
            tf.pow(score_nw_b, tf.kappa1),
            tf.add (
                tf.pow(score_n_b, tf.kappa1),
                tf.add (
                    tf.pow(score_ne_b, tf.kappa1), 
                    score_e_b)))

        

In [207]:
b=0x81
hib=[4,0,1,0,2,0,1,0,3,0,1,0,2,0,1,0]
lob=[4,3,2,2,1,1,1,1,0,0,0,0,0,0,0,0]
rh = (b & 0xF0) >> 4
rl = b & 0xF
hib[rh]+lob[rl]

6

In [236]:
def central_mask(b):
    """
    determine a mask covering the 'central' zero bits
    e.g. central_mask( 0 1 0 0 | 0 0 1 1 ) =
                       0 0 1 1 | 1 1 0 0 = 0x3C
    Args:
        b: an 8 bit integer representing a line of stones of one color
    Returns:
        a pair of tensors:
        1. A bit mask covering the central 'empty' region
        2. The length of that masked region.
    """
    hi_len  = tf.constant([4,0,1,0,2,0,1,0,3,0,1,0,2,0,1,0], dtype=tf.int32) 
    hi_mask = tf.constant(
        [0xF,0x0,0x1,0x0,0x3,0x0,0x1,0x0,0x7,0x0,0x1,0x0,0x3,0x0,0x1,0x0], 
        dtype=tf.int32) # hi byte

    lo_len  = tf.constant([4,3,2,2,1,1,1,1,0,0,0,0,0,0,0,0], dtype=tf.int32)
    lo_mask = tf.constant(
        [0xF,0xE,0xC,0xC,0x8,0x8,0x8,0x8,0x0,0x0,0x0,0x0,0x0,0x0,0x0,0x0], 
        dtype=tf.int32) # lo byte

    i_h = tf.bitwise.right_shift(tf.bitwise.bitwise_and(b, 0xF0), 4)
    i_l = tf.bitwise.bitwise_and(b, 0xF)

    return hi_mask[i_h]*0x10 + lo_mask[i_l], hi_len[i_h] + lo_len[i_l]

In [214]:
a=tf.constant([[1,2,3,4,5,6],[3,4,5,6,7,8], [2,3,4,5,6,7]], dtype=tf.int32)

In [209]:
s=a[:-1]

In [237]:
with tf.Session() as sess:
    print(sess.run(central_mask(tf.constant(0x43, dtype=tf.int32))))

(60, 4)
